# Librairie

In [ ]:
import tensorflow as tf 
import numpy as np 
import matplotlib.pyplot as plt 
import pandas as pd

In [ ]:
!unzip /content/drive/MyDrive/Projet_Jedha_DL/25_classes/train.zip

# Preprocessing Model

In [ ]:
def preprocessing_train_test():
    
    img_generator = tf.keras.preprocessing.image.ImageDataGenerator(#rotation_range=90,
                                                                brightness_range=(0.5,1), 
                                                                #shear_range=0.2, 
                                                                zoom_range=0.2,
                                                                channel_shift_range=0.2,
                                                                horizontal_flip=True,
                                                                vertical_flip=True,
                                                                rescale=1./255,
                                                                validation_split=0.2
                                                                )

    img_generator_flow_train = img_generator.flow_from_directory(directory="/content/train",
                                                             target_size=(150, 150),
                                                             batch_size=32,
                                                             shuffle=True,
                                                             subset="training"
                                                             )

    img_generator_flow_test = img_generator.flow_from_directory(directory="/content/train",
                                                               target_size=(150, 150),
                                                               batch_size=32,
                                                               shuffle=True,
                                                               subset="validation"
                                                               )
    return img_generator_flow_train, img_generator_flow_test

# Model

### InceptionV3

In [1]:
def inceptionV3(nbr_cat, train = False):
    
    base_model = tf.keras.applications.InceptionV3(input_shape=(150,150,3),
                                                include_top=False,
                                                weights = "imagenet"
                                                )
    
    if train == False:
        base_model.trainable = False
    else:
        base_model.trainable = True
        fine_tune_at = len(base_model.layers) - 30
        for layer in base_model.layers[:fine_tune_at]:
            layer.trainable = False
    
    model = tf.keras.Sequential([base_model,
                              tf.keras.layers.MaxPooling2D(),
                              tf.keras.layers.Flatten(),
                              tf.keras.layers.Dense(nbr_cat, activation="softmax")
                              ])
       
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate = 0.001),
              loss = tf.keras.losses.CategoricalCrossentropy(),
              metrics = [tf.keras.metrics.CategoricalAccuracy()])
    
    return model

### Xception 

In [5]:
def Xception(nbr_cat, train = False):
    
    base_model = tf.keras.applications.Xception(input_shape=(150,150,3),
                                                include_top=False,
                                                weights = "imagenet"
                                                )
    
    if train == False:
        base_model.trainable = False
    else:
        base_model.trainable = True
        fine_tune_at = len(base_model.layers) - 30
        for layer in base_model.layers[:fine_tune_at]:
            layer.trainable = False
    
    model = tf.keras.Sequential([base_model,
                              tf.keras.layers.GlobalAveragePooling2D(),
                              tf.keras.layers.Dropout(0.2),
                              tf.keras.layers.Dense(nbr_cat, activation="softmax")
                              ])
       
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate = 0.001),
              loss = tf.keras.losses.CategoricalCrossentropy(),
              metrics = [tf.keras.metrics.CategoricalAccuracy()])
    
    return model

# Prediction

In [6]:
def resize(image_path):
    """
    IN : localisation de l'image
    OUT: l'image redimensionné
    obj: redimensionner l'igame avant prediction
    """
    image_resize = tf.io.read_file(image_path) # read the file as byte type
    image_resize = tf.image.decode_image(image_resize, channels=3) # convert bytes to a tensor
    image_resize = tf.image.convert_image_dtype(image_resize, tf.float64) # convert the image tensor from int 0-255 to float 0-1
    image_resize = tf.image.resize(image_resize, [150, 150]) # resize the image so the model can run inference
    image_resize = tf.expand_dims(image_resize, axis=0) # let's expand the dimension so the input data has shape (1,299,299,3)
    return image_resize

In [1]:
def prediction(model, image):
    """
    IN: model est le model chargé avec les poids; image est l'image redimensionné par def resize
    OUT: la valeur de la prediction, c'est un integer
    obj: appliquer le model à une image pour sortir une prediction
    """
    pred = model.predict(image)
    pred_label = tf.argmax(pred, axis=-1).numpy()
    return pred_label[0] # On obtien un array avec la prediction, je viens chercher la valeur de cette prediction


In [ ]:
def categorie(pred):
    """
    IN: integer de def prediction()
    OUT: Le nom de la catégorie de la prediction
    obj: Rechercher dans le  fichier csv qui contient les labels et les catégories. Sortir la catégorie à laquelle l'image appartient
    """
    labels_df = pd.read_csv("labels.csv", sep=";")
    label = labels_df.iloc[pred,:]['categorie']
    return label

In [ ]:
def dictionnaire():
    """
    Cette définition retourne un dictionnaire contenant 12 clefs qui représente 12 lettres. Elles ne contiennent pas de valeurs car elle seront remplacés par les lettres de la prédiction.
    Cette def permet de remettre à 0 le dictionnaire à chaque nouvelle prédiction.
    """
    return {"l0" : "","l1" : "","l2" : "","l3" : "","l4" : "","l5" : "","l6" : "","l7" : "","l8" : "","l9" : "","l10" : "","l11" : ""}